<img src="http://oproject.org/img/ROOT.png" height="30%" width="30%">
<img src="http://oproject.org/img/tmvalogo.png" height="30%" width="30%">

<hr style="border-top-width: 4px; border-top-color: #34609b;">

# DNN Example

## Declare Factory

In [1]:
from ROOT import TMVA, TFile, TTree, TCut, TString

Welcome to JupyROOT 6.18/00


In [2]:
TMVA.Tools.Instance()

inputFile1 = TFile.Open("http://opendata.atlas.cern/release/samples/MC/mc_161005.ggH125_WW2lep.root")
inputFile2= TFile.Open("http://opendata.atlas.cern/release/samples/MC/mc_105985.WW.root")
outputFile = TFile.Open("TMVAOutputDNN.root", "RECREATE")

factory = TMVA.Factory("TMVAClassification", outputFile,
        "!V:!Silent:Color:!DrawProgressBar:AnalysisType=Classification" )

## Declare Variables in DataLoader

In [3]:
loader = TMVA.DataLoader("dataset_dnn")

loader.AddVariable("jet_n")
loader.AddVariable("lep_pt[0]")
loader.AddVariable("lep_pt[1]")
loader.AddVariable("lep_eta[0]")
loader.AddVariable("lep_eta[1]")
loader.AddVariable("jet_pt[0]")
loader.AddVariable("jet_pt[1]")
#loader.AddVariable("jet_pt[1]")
#loader.AddVariable("jet_pt[1]")
#loader.AddVariable("jet_pt[1]")
#loader.AddVariable("jet_pt[1]")
#loader.AddVariable("jet_pt[1]")
#loader.AddVariable("jet_pt[1]")
#loader.AddVariable("jet_pt[1]")




#loader.AddVariable("var5 := var1-var3")
#loader.AddVariable("var6 := var1+var2")

## Setup Dataset(s)

In [4]:
tsignal = inputFile1.Get("mini")
tbackground = inputFile2.Get("mini")

loader.AddSignalTree(tsignal)
loader.AddBackgroundTree(tbackground) 
loader.PrepareTrainingAndTestTree(TCut(""),
        "nTrain_Signal=2000:nTrain_Background=2000:SplitMode=Random:NormMode=NumEvents:!V")

DataSetInfo              : [dataset_dnn] : Added class "Signal"
                         : Add Tree mini of type Signal with 100000 events
DataSetInfo              : [dataset_dnn] : Added class "Background"
                         : Add Tree mini of type Background with 500000 events
                         : Dataset[dataset_dnn] : Class index : 0  name : Signal
                         : Dataset[dataset_dnn] : Class index : 1  name : Background


# Configure Network Layout 

In [5]:
# General layout
layoutString = TString("Layout=TANH|128,TANH|128,TANH|128,LINEAR");

# Training strategies
training0 = TString("LearningRate=1e-1,Momentum=0.9,Repetitions=1,"
                        "ConvergenceSteps=2,BatchSize=256,TestRepetitions=10,"
                        "WeightDecay=1e-4,Regularization=L2,"
                        "DropConfig=0.0+0.5+0.5+0.5, Multithreading=True")
training1 = TString("LearningRate=1e-2,Momentum=0.9,Repetitions=1,"
                        "ConvergenceSteps=2,BatchSize=256,TestRepetitions=10,"
                        "WeightDecay=1e-4,Regularization=L2,"
                        "DropConfig=0.0+0.0+0.0+0.0, Multithreading=True")
trainingStrategyString = TString("TrainingStrategy=")
trainingStrategyString += training0 + TString("|") + training1

# General Options
dnnOptions = TString("!H:!V:ErrorStrategy=CROSSENTROPY:VarTransform=N:"
        "WeightInitialization=XAVIERUNIFORM")
dnnOptions.Append(":")
dnnOptions.Append(layoutString)
dnnOptions.Append(":")
dnnOptions.Append(trainingStrategyString)

'!H:!V:ErrorStrategy=CROSSENTROPY:VarTransform=N:WeightInitialization=XAVIERUNIFORM:Layout=TANH|128,TANH|128,TANH|128,LINEAR:TrainingStrategy=LearningRate=1e-1,Momentum=0.9,Repetitions=1,ConvergenceSteps=2,BatchSize=256,TestRepetitions=10,WeightDecay=1e-4,Regularization=L2,DropConfig=0.0+0.5+0.5+0.5, Multithreading=True|LearningRate=1e-2,Momentum=0.9,Repetitions=1,ConvergenceSteps=2,BatchSize=256,TestRepetitions=10,WeightDecay=1e-4,Regularization=L2,DropConfig=0.0+0.0+0.0+0.0, Multithreading=True'

# Booking Methods

In [6]:
# Standard implementation, no dependencies.
stdOptions =  dnnOptions + ":Architecture=CPU"
factory.BookMethod(loader, TMVA.Types.kDNN, "DNN", stdOptions)

# CPU implementation, using BLAS
#cpuOptions = dnnOptions + ":Architecture=CPU"
#factory.BookMethod(loader, TMVA.Types.kDNN, "DNN CPU", cpuOptions)

<ROOT.TMVA::MethodDNN object ("DNN") at 0x65919e0>

Factory                  : Booking method: DNN
                         : 
DNN                      : [dataset_dnn] : Create Transformation "N" with events from all classes.
                         : 
                         : Transformation, Variable selection : 
                         : Input : variable 'jet_n' <---> Output : variable 'jet_n'
                         : Input : variable 'lep_pt[0]' <---> Output : variable 'lep_pt[0]'
                         : Input : variable 'lep_pt[1]' <---> Output : variable 'lep_pt[1]'
                         : Input : variable 'lep_eta[0]' <---> Output : variable 'lep_eta[0]'
                         : Input : variable 'lep_eta[1]' <---> Output : variable 'lep_eta[1]'
                         : Input : variable 'jet_pt[0]' <---> Output : variable 'jet_pt[0]'
                         : Input : variable 'jet_pt[1]' <---> Output : variable 'jet_pt[1]'
<WARNING>                : Expression: jet_pt[0] does not provide data for this event. This e

## Train Methods

In [7]:
factory.TrainAllMethods()

Factory                  : Train all methods
Factory                  : [dataset_dnn] : Create Transformation "I" with events from all classes.
                         : 
                         : Transformation, Variable selection : 
                         : Input : variable 'jet_n' <---> Output : variable 'jet_n'
                         : Input : variable 'lep_pt[0]' <---> Output : variable 'lep_pt[0]'
                         : Input : variable 'lep_pt[1]' <---> Output : variable 'lep_pt[1]'
                         : Input : variable 'lep_eta[0]' <---> Output : variable 'lep_eta[0]'
                         : Input : variable 'lep_eta[1]' <---> Output : variable 'lep_eta[1]'
                         : Input : variable 'jet_pt[0]' <---> Output : variable 'jet_pt[0]'
                         : Input : variable 'jet_pt[1]' <---> Output : variable 'jet_pt[1]'
TFHandler_Factory        :   Variable          Mean          RMS   [        Min          Max ]
                         : -

## Test and Evaluate Methods

In [8]:
factory.TestAllMethods()
factory.EvaluateAllMethods()

Factory                  : Test all methods
Factory                  : Test method: DNN for Classification performance
                         : 
DNN                      : [dataset_dnn] : Evaluation of DNN on testing sample (17296 events)
                         : Elapsed time for evaluation of 17296 events: 1.03 sec       
Factory                  : Evaluate all methods
Factory                  : Evaluate classifier: DNN
                         : 
TFHandler_DNN            :   Variable          Mean          RMS   [        Min          Max ]
                         : ---------------------------------------------------------------------
                         :      jet_n:    -0.41433     0.32478   [     -1.0000      2.0000 ]
                         :  lep_pt[0]:    -0.90082     0.10214   [     -1.0000     0.76807 ]
                         :  lep_pt[1]:    -0.91664    0.083682   [     -1.0000      1.0647 ]
                         : lep_eta[0]:   0.0018202     0.47653   [     -

## Plot ROC Curve
We enable JavaScript visualisation for the plots

In [9]:
%jsroot on

In [10]:
c = factory.GetROCCurve(loader)
c.Draw()